In [1]:
import tensorflow
print(tensorflow.__version__)

2.13.0


In [2]:
# !pip install stable-baselines3[extra]

In [3]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import time
import random
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm

# from baselines.ppo2 import ppo2
from baselines.common.vec_env.dummy_vec_env import DummyVecEnv

# from baselines import deepq  # Deep Q Network (DQN) 
# from baselines import bench  # https://github.com/openai/baselines/tree/ea25b9e8b234e6ee1bca43083f8f3cf974143998/baselines/bench
# from baselines import logger
# import tensorflow as tf

from baselines.common.tf_util import make_session

# import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import DQN


In [4]:
# Function to read images from a directory and return image paths and labels
def read_images_from_directory(directory):
    images = []
    labels = []
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        if os.path.isdir(label_dir):
            for image_file in os.listdir(label_dir):
                image_path = os.path.join(label_dir, image_file)
                images.append(image_path)
                labels.append(label)
    return images, labels

In [5]:
# Directories for train and test sets
train_directory = r"C:\Users\123\Documents\SD Labs\Cyber\dataset\OCT2017\train"
test_directory = r"C:\Users\123\Documents\SD Labs\Cyber\dataset\OCT2017\test"


In [6]:
# Perform exploratory data analysis (EDA)
def perform_eda(images, labels):
    # Count the number of images in each class
    class_counts = {label: labels.count(label) for label in set(labels)}
    print("Class Counts:", class_counts)

    # Show sample images from each class
    num_samples_per_class = 3
    fig, axes = plt.subplots(len(class_counts), num_samples_per_class, figsize=(12, 8))
    fig.suptitle("Sample Images from Each Class", fontsize=16)

    for i, (label, count) in enumerate(class_counts.items()):
        image_paths = [img_path for img_path, img_label in zip(images, labels) if img_label == label]
        for j in range(num_samples_per_class):
            img = Image.open(image_paths[j])
            axes[i, j].imshow(img)
            axes[i, j].axis("off")
            axes[i, j].set_title(f"{label} - {os.path.basename(image_paths[j])}")

    plt.show()

In [7]:

# Read train and test images along with their labels
train_images, train_labels = read_images_from_directory(train_directory)
test_images, test_labels = read_images_from_directory(test_directory)

In [8]:

# Perform EDA on train set
# perform_eda(train_images, train_labels)

# Perform EDA on test set
# perform_eda(test_images, test_labels)


In [9]:
# !pip install git+https://github.com/openai/baselines.git@ea25b9e8b234e6ee1bca43083f8f3cf974143998

In [10]:
# Model / data parameters
num_classes = 4
input_shape = (28, 28, 1)

In [11]:


# # Convert train_images to a numpy array
# train_images = np.array(train_images)

# # Scale images to the [0, 1] range
# train_images = train_images.astype("float32") / 255

# # Make sure images have shape (28, 28, 1)
# train_images = np.expand_dims(train_images, -1)

# print("train_images shape:", train_images.shape)
# print(train_images.shape[0], "train_images samples")


In [14]:
# Function to load and preprocess images
def load_and_preprocess_images(file_paths, percent=100):
    images = []
    num_images = int(len(file_paths) * (percent / 100))
    for file_path in tqdm(file_paths[:num_images]):
        img = Image.open(file_path)
        img = img.resize((28, 28))  # Resize the image to (28, 28)
        img = np.array(img)
        images.append(img)
    images = np.array(images)
    images = images.astype("float32") / 255
    images = np.expand_dims(images, -1)
    return images

# Convert train_images and test_images to numpy arrays and preprocess them
train_images = load_and_preprocess_images(train_images,  percent=10)
test_images = load_and_preprocess_images(test_images,  percent=10)

print("train_images shape:", train_images.shape)
print(train_images.shape[0], "train_images samples")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:15<00:00,  6.50it/s]

train_images shape: (8348, 28, 28, 1)
8348 train_images samples


In [17]:
# convert class vectors to binary class matrices
# y_train_one_hot = keras.utils.to_categorical(test_images, num_classes)
# y_test_one_hot = keras.utils.to_categorical(test_labels, num_classes)


# Convert class labels to integer class indices
class_indices = {label: index for index, label in enumerate(set(train_labels))}
train_labels = [class_indices[label] for label in train_labels]
test_labels = [class_indices[label] for label in test_labels]

# convert class vectors to binary class matrices
y_train_one_hot = keras.utils.to_categorical(train_labels, num_classes)
y_test_one_hot = keras.utils.to_categorical(test_labels, num_classes)

In [ ]:
# def keras_train(batch_size=32, epochs=2):
#     model = keras.Sequential(
#         [
#             keras.Input(shape=input_shape),
#             layers.Flatten(),
#             layers.Dense(64, activation='relu'),
#             layers.Dense(64, activation='relu'),
#             layers.Dense(num_classes, activation="softmax")
#         ]
#     )

#     model.summary()

#     model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

#     start_time = time.time()
#     model.fit(x_train, y_train_one_hot, batch_size=batch_size, epochs=epochs, validation_split=0.1)
#     end_time = time.time()

#     score = model.evaluate(x_test, y_test_one_hot, verbose=0)
#     print("Test loss:", score[0])
#     print("Test accuracy:", score[1])
#     print("Training Time:", end_time - start_time)

# keras_train()

In [ ]:
# class MnistEnv(gym.Env):
#     def __init__(self, images_per_episode=1, dataset=(x_train, y_train), random=True):
#         super().__init__()

#         self.action_space = gym.spaces.Discrete(10)
#         self.observation_space = gym.spaces.Box(low=0, high=1,
#                                                 shape=(28, 28, 1),
#                                                 dtype=np.float32)

#         self.images_per_episode = images_per_episode
#         self.step_count = 0

#         self.x, self.y = dataset
#         self.random = random
#         self.dataset_idx = 0

#     def step(self, action):
#         done = False
#         reward = int(action == self.expected_action)

#         obs = self._next_obs()

#         self.step_count += 1
#         if self.step_count >= self.images_per_episode:
#             done = True

#         return obs, reward, done, {}

#     def reset(self):
#         self.step_count = 0

#         obs = self._next_obs()
#         return obs

#     def _next_obs(self):
#         if self.random:
#             next_obs_idx = random.randint(0, len(self.x) - 1)
#             self.expected_action = int(self.y[next_obs_idx])
#             obs = self.x[next_obs_idx]

#         else:
#             obs = self.x[self.dataset_idx]
#             self.expected_action = int(self.y[self.dataset_idx])

#             self.dataset_idx += 1
#             if self.dataset_idx >= len(self.x):
#                 raise StopIteration()

#         return obs

In [ ]:
# def mnist_dqn():
#     logger.configure(dir='./logs/mnist_dqn', format_strs=['stdout', 'tensorboard'])
#     env = MnistEnv(images_per_episode=1)
#     env = bench.Monitor(env, logger.get_dir())

#     model = deepq.learn(
#         env,
#         "mlp",
#         num_layers=1,
#         num_hidden=64,
#         activation=tf.nn.relu,
#         hiddens=[32],
#         dueling=True,
#         lr=1e-4,
#         total_timesteps=int(1.2e5),
#         buffer_size=10000,
#         exploration_fraction=0.1,
#         exploration_final_eps=0.01,
#         train_freq=4,
#         learning_starts=10000,
#         target_network_update_freq=1000,
#     )

#     model.save('dqn_mnist.pkl')
#     env.close()

#     return model

# start_time = time.time()
# dqn_model = mnist_dqn()
# print("DQN Training Time:", time.time() - start_time)


In [ ]:
# def mnist_dqn_eval(dqn_model):
#     attempts, correct = 0,0

#     env = MnistEnv(images_per_episode=1, dataset=(x_test, y_test), random=False)

#     try:
#         while True:
#             obs, done = env.reset(), False
#             while not done:
#                 obs, rew, done, _ = env.step(dqn_model(obs[None])[0])

#                 attempts += 1
#                 if rew > 0:
#                     correct += 1

#     except StopIteration:
#         print()
#         print('validation done...')
#         print('Accuracy: {0}%'.format((float(correct) / attempts) * 100))

# mnist_dqn_eval(dqn_model)

In [ ]:
# def mnist_ppo():
#     logger.configure(dir='./logs/mnist_ppo', format_strs=['stdout', 'tensorboard'])
#     env = DummyVecEnv([lambda: bench.Monitor(MnistEnv(images_per_episode=1), logger.get_dir())])

#     model = ppo2.learn(
#         env=env,
#         network='mlp',
#         num_layers=2,
#         num_hidden=64,
#         nsteps=32,
#         total_timesteps=int(1.2e5),
#         seed=int(time.time()))

#     return model

# start_time = time.time()
# ppo_model = mnist_ppo()
# print("PPO Training Time:", time.time() - start_time)

In [ ]:
# def mnist_ppo_eval(ppo_model):
#     attempts, correct = 0,0

#     env = DummyVecEnv([lambda: MnistEnv(images_per_episode=1, dataset=(x_test, y_test), random=False)])

#     try:
#         while True:
#             obs, done = env.reset(), [False]
#             while not done[0]:
#                 obs, rew, done, _ = env.step(ppo_model.step(obs[None])[0])

#                 attempts += 1
#                 if rew[0] > 0:
#                     correct += 1

#     except StopIteration:
#         print()
#         print('validation done...')
#         print('Accuracy: {0}%'.format((float(correct) / attempts) * 100))

# mnist_ppo_eval(ppo_model)